# 基于 SAM 的多模态医学图像分割入门教程

> SAM（Segment Anything Model）通常使用“提示（prompt）”进行分割。

本教程教你：
1. 如何准备数据（图像 + mask + 元数据）
2. 如何把 mask 转成 prompt（例如 bbox）
3. 如何微调 SAM 的 mask decoder

**注意**：SAM 很大，真正训练需要 GPU。为了可跑通，我们做一个**极简可运行版**。


## 0. 环境准备

```bash
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
pip install numpy pandas pillow matplotlib tqdm
pip install git+https://github.com/facebookresearch/segment-anything.git
```

下载 SAM 权重（任选一个）：
```bash
# 这里示例 vit_b，文件较小
wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth -O sam_vit_b.pth
```


## 1. 数据结构

```
examples/transformer_tutorial2/
  sam_multimodal_seg.ipynb
  data/
    sam_tutorial/
      images/
      masks/
      metadata.csv
```

我们先创建一个小样例数据集。


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw

ROOT = Path('examples/transformer_tutorial2/data/sam_tutorial')
IMG_DIR = ROOT / 'images'
MSK_DIR = ROOT / 'masks'
ROOT.mkdir(parents=True, exist_ok=True)
IMG_DIR.mkdir(parents=True, exist_ok=True)
MSK_DIR.mkdir(parents=True, exist_ok=True)

rows = []
for i in range(10):
    img = Image.new('RGB', (256, 256), color=(0, 0, 0))
    mask = Image.new('L', (256, 256), color=0)

    draw_img = ImageDraw.Draw(img)
    draw_msk = ImageDraw.Draw(mask)

    cx, cy = np.random.randint(60, 196, size=2)
    r = np.random.randint(20, 45)
    bbox = (cx - r, cy - r, cx + r, cy + r)
    draw_img.ellipse(bbox, fill=(180, 180, 180))
    draw_msk.ellipse(bbox, fill=255)

    image_id = f'sample_{i:03d}'
    img.save(IMG_DIR / f'{image_id}.png')
    mask.save(MSK_DIR / f'{image_id}.png')

    rows.append({
        'image_id': image_id,
        'age': int(np.random.randint(18, 80)),
        'sex': int(np.random.randint(0, 2)),
        'modality': int(np.random.randint(0, 3)),
    })

meta = pd.DataFrame(rows)
meta.to_csv(ROOT / 'metadata.csv', index=False)
meta.head()


## 2. 数据集与 prompt（bbox）

SAM 的输入除了图像，还需要 prompt（比如 bbox 或点）。
我们从 mask 中计算 bbox，作为训练 prompt。


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class SamSegDataset(Dataset):
    def __init__(self, root_dir, img_size=256):
        self.root_dir = Path(root_dir)
        self.img_dir = self.root_dir / 'images'
        self.msk_dir = self.root_dir / 'masks'
        self.meta = pd.read_csv(self.root_dir / 'metadata.csv')

        self.transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.meta)

    def __getitem__(self, idx):
        row = self.meta.iloc[idx]
        image_id = row['image_id']

        # 读取图像（RGB）
        img = Image.open(self.img_dir / f'{image_id}.png').convert('RGB')
        # 读取 mask（灰度）
        mask = Image.open(self.msk_dir / f'{image_id}.png').convert('L')

        img = self.transform(img)
        mask = self.transform(mask)
        mask = (mask > 0.5).float()

        # 计算 bbox（x0,y0,x1,y1）
        # 从 mask 里找出前景像素，计算 bbox
        ys, xs = torch.where(mask[0] > 0)
        x0, x1 = xs.min().item(), xs.max().item()
        y0, y1 = ys.min().item(), ys.max().item()
        bbox = torch.tensor([x0, y0, x1, y1], dtype=torch.float32)

        # 把元数据拼成向量
        meta = torch.tensor([row['age'], row['sex'], row['modality']], dtype=torch.float32)
        meta[0] = meta[0] / 100.0
        meta[2] = meta[2] / 2.0

        return img, mask, bbox, meta

DATASET_ROOT = 'examples/transformer_tutorial2/data/sam_tutorial'
dataset = SamSegDataset(DATASET_ROOT)
img, mask, bbox, meta = dataset[0]
img.shape, mask.shape, bbox, meta


## 2.1 快速运行自检（确保数据与 prompt 正常）

这一小段不会训练，只做**快速检查**：
- 数据能读取
- prompt (bbox) 正常


In [ ]:
# ✅ 快速自检：数据 + bbox 生成
img, mask, bbox, meta = dataset[0]
print('image shape:', img.shape)
print('bbox:', bbox.tolist())
assert bbox.numel() == 4, 'bbox 需要 4 个值 (x0,y0,x1,y1)'


## 3. 构建 SAM 微调模型（只微调 mask decoder）

SAM 结构非常大，我们做一个“轻量微调”：
- 冻结图像编码器与 prompt 编码器
- 只训练 mask decoder

同时，我们加入一个简单的元数据嵌入，把它加到 mask decoder 的输出上。


In [ ]:
import torch.nn as nn
from pathlib import Path
from segment_anything import sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide

class SamWithMeta(nn.Module):
    def __init__(self, checkpoint_path=None, meta_dim=3, model_type='vit_b'):
        super().__init__()
        self.sam = sam_model_registry[model_type](checkpoint=checkpoint_path)
        self.transform = ResizeLongestSide(self.sam.image_encoder.img_size)

        # 冻结编码器，只训练 mask decoder
        for p in self.sam.image_encoder.parameters():
            p.requires_grad = False
        for p in self.sam.prompt_encoder.parameters():
            p.requires_grad = False

        self.meta_mlp = nn.Sequential(
            nn.Linear(meta_dim, 64), nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, image, bbox, meta):
        # image: [B,3,H,W] -> SAM 需要先 resize/pad 到 1024
        if bbox.dim() == 1:
            bbox = bbox.unsqueeze(0)

        # resize image 和 box
        resized_image = self.transform.apply_image_torch(image)
        resized_image = self.sam.preprocess(resized_image)

        resized_boxes = self.transform.apply_boxes_torch(bbox, image.shape[-2:])

        # 编码图像和 prompt
        image_embedding = self.sam.image_encoder(resized_image)
        sparse_embeddings, dense_embeddings = self.sam.prompt_encoder(
            points=None,
            boxes=resized_boxes,
            masks=None,
        )

        # mask decoder
        low_res_masks, _ = self.sam.mask_decoder(
            image_embeddings=image_embedding,
            image_pe=self.sam.prompt_encoder.get_dense_pe(),
            sparse_prompt_embeddings=sparse_embeddings,
            dense_prompt_embeddings=dense_embeddings,
            multimask_output=False,
        )

        # 结合元数据：学一个 bias 加到 logits 上
        meta_bias = self.meta_mlp(meta).view(-1, 1, 1, 1)
        low_res_masks = low_res_masks + meta_bias
        return low_res_masks

CHECKPOINT = Path('sam_vit_b.pth')
checkpoint_path = CHECKPOINT if CHECKPOINT.exists() else None
if checkpoint_path is None:
    print('⚠️ 未找到 sam_vit_b.pth，将使用随机初始化权重（仅用于跑通流程）。')

model = SamWithMeta(checkpoint_path=checkpoint_path)


## 4. 训练准备与训练循环（简化版）


In [ ]:
from torch.optim import Adam

def dice_loss(logits, targets, eps=1e-6):
    probs = torch.sigmoid(logits)
    num = 2 * (probs * targets).sum(dim=(1,2,3))
    den = probs.sum(dim=(1,2,3)) + targets.sum(dim=(1,2,3)) + eps
    return 1 - (num / den).mean()

loader = DataLoader(dataset, batch_size=1, shuffle=True)
optimizer = Adam(model.parameters(), lr=1e-4)

model.train()
for epoch in range(1):
    epoch_loss = 0
    for img, mask, bbox, meta in loader:
        # 前向传播
        logits = model(img, bbox, meta)
        # SAM 输出是低分辨率，需要上采样到 mask 大小
        logits = nn.functional.interpolate(logits, size=mask.shape[-2:], mode='bilinear', align_corners=False)

        # 计算损失（BCE + Dice）
        loss = nn.functional.binary_cross_entropy_with_logits(logits, mask) + dice_loss(logits, mask)
        # 反向传播与更新参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print('avg loss:', epoch_loss / len(loader))


## 5. 评估与可视化


In [ ]:
import matplotlib.pyplot as plt

model.eval()
with torch.no_grad():
    img, mask, bbox, meta = dataset[0]
    logits = model(img.unsqueeze(0), bbox.unsqueeze(0), meta.unsqueeze(0))
    logits = nn.functional.interpolate(logits, size=mask.shape[-2:], mode='bilinear', align_corners=False)
    pred = torch.sigmoid(logits)[0,0]

    plt.figure(figsize=(8,3))
    plt.subplot(1,3,1); plt.title('Image'); plt.imshow(img.permute(1,2,0))
    plt.subplot(1,3,2); plt.title('GT Mask'); plt.imshow(mask[0], cmap='gray')
    plt.subplot(1,3,3); plt.title('Pred'); plt.imshow(pred, cmap='gray')
    plt.show()


## 6. 用你自己的数据怎么做？

1. 替换 `images/`、`masks/` 和 `metadata.csv`。
2. 如果你有**真实的 prompt**（例如医生给出的点/框），可以直接用它们。
3. 如果你没有 prompt，可以像本教程一样从 mask 生成 bbox。

### 预训练与微调
- SAM 本身是大模型，通常只微调 mask decoder。
- 你也可以逐步解冻 image encoder（学习率调小）。
